# Momentum Transformer (Long-only, Weight=1, 5bps)

このNotebookは、論文 **Trading with the Momentum Transformer** と著者実装を参考に、`data/prices.csv` へ **Momentum Transformer (Decoder-Only TFT 風)** を適用した実験です。

前提と制約:
- ロングオンリー（ショート禁止）
- 日次の合計ウェイト=1（レバレッジ禁止）
- 取引コストは **5 bps** のみ
- 未来情報リークを避ける（bfill 不使用、スケーリングは訓練データのみ）

出力先:
- 図: `outputs/figures/`
- テーブル: `outputs/tables/`
- ログ: `outputs/logs/`


In [ ]:
import os
import math
import random
from pathlib import Path

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler

# ==== 実行環境・出力設定 ====
BASE_DIR = Path('/Users/kencharoff/workspace/projects/momentum/momentum-transformer')
DATA_PATH = BASE_DIR / 'data' / 'prices.csv'
OUTPUT_DIR = BASE_DIR / 'outputs'
FIG_DIR = OUTPUT_DIR / 'figures'
TABLE_DIR = OUTPUT_DIR / 'tables'
LOG_DIR = OUTPUT_DIR / 'logs'

FIG_DIR.mkdir(parents=True, exist_ok=True)
TABLE_DIR.mkdir(parents=True, exist_ok=True)
LOG_DIR.mkdir(parents=True, exist_ok=True)

os.environ['MPLCONFIGDIR'] = str(LOG_DIR / 'mpl')
import matplotlib.pyplot as plt
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 再現性
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# 実験パラメータ
TIME_STEPS = 252
TRAIN_VALID_RATIO = 0.90
TEST_BOUNDARY = 2020
TEST_END = 2026
VOL_TARGET = 0.15
VOL_LOOKBACK = 60
HALFLIFE_WINSORISE = 252
VOL_THRESHOLD = 5

BATCH_SIZE = 128
EPOCHS = 50
EARLY_STOPPING = 10
HIDDEN_SIZE = 20
DROPOUT_RATE = 0.1
NUM_HEADS = 4
LEARNING_RATE = 1e-3
MAX_GRAD_NORM = 1.0

# 大きい場合は学習サンプル数を制限可能（Noneで全件）
MAX_TRAIN_WINDOWS = None


In [ ]:
# ==== データ読み込み（Adj Close 使用） ====
prices = pd.read_csv(DATA_PATH, header=[0,1], index_col=0, parse_dates=True)
adj = prices['Adj Close'].copy()
adj.index = pd.to_datetime(adj.index)

long_df = adj.stack().rename('close').reset_index()
long_df.columns = ['date', 'ticker', 'close']
long_df = long_df.sort_values(['ticker', 'date']).reset_index(drop=True)

print(long_df.head())
print(long_df['ticker'].nunique(), 'tickers')
print(long_df['date'].min(), long_df['date'].max())


## 特徴量作成（著者実装準拠・リーク防止）
- winsorize（EWMA mean/std）
- ボラティリティ調整
- 正規化リターン（1/21/63/126/252日）
- MACD（8/24, 16/48, 32/96）


In [ ]:
def winsorize_series(srs: pd.Series) -> pd.Series:
    ewm = srs.ewm(halflife=HALFLIFE_WINSORISE, min_periods=HALFLIFE_WINSORISE)
    mean = ewm.mean()
    std = ewm.std()
    upper = mean + VOL_THRESHOLD * std
    lower = mean - VOL_THRESHOLD * std
    return srs.clip(lower, upper)


def calc_returns(srs: pd.Series, day_offset: int = 1) -> pd.Series:
    return srs / srs.shift(day_offset) - 1.0


def calc_daily_vol(daily_returns: pd.Series) -> pd.Series:
    # bfillは使わず、初期欠損はdrop
    return daily_returns.ewm(span=VOL_LOOKBACK, min_periods=VOL_LOOKBACK).std()


def calc_vol_scaled_returns(daily_returns: pd.Series, daily_vol: pd.Series) -> pd.Series:
    annualised_vol = daily_vol * np.sqrt(252)
    return daily_returns * VOL_TARGET / annualised_vol.shift(1)


def calc_normalised_returns(srs: pd.Series, daily_vol: pd.Series, day_offset: int) -> pd.Series:
    return calc_returns(srs, day_offset) / (daily_vol * np.sqrt(day_offset))


def macd_signal(srs: pd.Series, short_ts: int, long_ts: int) -> pd.Series:
    def _hl(ts):
        return np.log(0.5) / np.log(1 - 1 / ts)
    macd = (
        srs.ewm(halflife=_hl(short_ts), min_periods=short_ts).mean()
        - srs.ewm(halflife=_hl(long_ts), min_periods=long_ts).mean()
    )
    q = macd / srs.rolling(63, min_periods=63).std()
    q = q / q.rolling(252, min_periods=252).std()
    return q


def build_features(df_long: pd.DataFrame) -> pd.DataFrame:
    dfs = []
    for ticker, g in df_long.groupby('ticker'):
        g = g.sort_values('date').copy()
        g = g.set_index('date')
        srs = g['close']
        srs_w = winsorize_series(srs)

        daily_returns = calc_returns(srs_w, 1)
        daily_vol = calc_daily_vol(daily_returns)
        vol_scaled = calc_vol_scaled_returns(daily_returns, daily_vol)
        target_returns = vol_scaled.shift(-1)

        df = pd.DataFrame(index=g.index)
        df['ticker'] = ticker
        df['close'] = srs
        df['srs'] = srs_w
        df['daily_returns'] = daily_returns
        df['daily_vol'] = daily_vol
        df['target_returns'] = target_returns

        df['norm_daily_return'] = calc_normalised_returns(srs_w, daily_vol, 1)
        df['norm_monthly_return'] = calc_normalised_returns(srs_w, daily_vol, 21)
        df['norm_quarterly_return'] = calc_normalised_returns(srs_w, daily_vol, 63)
        df['norm_biannual_return'] = calc_normalised_returns(srs_w, daily_vol, 126)
        df['norm_annual_return'] = calc_normalised_returns(srs_w, daily_vol, 252)

        df['macd_8_24'] = macd_signal(srs_w, 8, 24)
        df['macd_16_48'] = macd_signal(srs_w, 16, 48)
        df['macd_32_96'] = macd_signal(srs_w, 32, 96)

        # ベースライン用
        df['raw_annual_return'] = calc_returns(srs_w, 252)
        df['macd_score'] = (df['macd_8_24'] + df['macd_16_48'] + df['macd_32_96']) / 3.0

        df['year'] = df.index.year
        df = df.dropna()
        dfs.append(df.reset_index())

    return pd.concat(dfs, ignore_index=True)

features = build_features(long_df)
print(features.head())
print(features['ticker'].nunique(), 'tickers after feature build')
print(features['date'].min(), features['date'].max())


## 固定分割（訓練・検証・テスト）
- `year < 2020`: train/valid
- `2020 <= year < 2026`: test


In [ ]:
feature_cols = [
    'norm_daily_return',
    'norm_monthly_return',
    'norm_quarterly_return',
    'norm_biannual_return',
    'norm_annual_return',
    'macd_8_24',
    'macd_16_48',
    'macd_32_96',
]

target_col = 'target_returns'

trainvalid = features[features['year'] < TEST_BOUNDARY].copy()
test = features[(features['year'] >= TEST_BOUNDARY) & (features['year'] < TEST_END)].copy()

train_parts = []
valid_parts = []
for ticker, g in trainvalid.groupby('ticker'):
    g = g.sort_values('date')
    split_idx = int(len(g) * TRAIN_VALID_RATIO)
    train_parts.append(g.iloc[:split_idx])
    valid_parts.append(g.iloc[split_idx:])

train = pd.concat(train_parts, ignore_index=True)
valid = pd.concat(valid_parts, ignore_index=True)

scaler = StandardScaler()
scaler.fit(train[feature_cols].values)

for df_ in (train, valid, test):
    df_[feature_cols] = scaler.transform(df_[feature_cols].values)

print(train[['date','ticker']].min())
print(train[['date','ticker']].max())
print(valid[['date','ticker']].min())
print(valid[['date','ticker']].max())
print(test[['date','ticker']].min())
print(test[['date','ticker']].max())


trainvalid_scaled = pd.concat([train, valid], ignore_index=True)

## スライディング窓の作成
- `TIME_STEPS=252`
- 予測は窓の最終時点（`t`）に対する `t+1` のリターン


In [ ]:
def make_sliding_windows(df: pd.DataFrame, time_steps: int, feature_cols: list, target_col: str, with_meta: bool = False):
    X_list, y_list, meta_list = [], [], []
    for ticker, g in df.groupby('ticker'):
        g = g.sort_values('date')
        X = g[feature_cols].to_numpy(dtype=np.float32)
        y = g[target_col].to_numpy(dtype=np.float32)
        dates = g['date'].to_numpy()
        if len(g) < time_steps:
            continue
        for i in range(len(g) - time_steps + 1):
            X_list.append(X[i:i+time_steps])
            y_list.append(y[i:i+time_steps])
            if with_meta:
                meta_list.append((ticker, dates[i+time_steps-1], y[i+time_steps-1]))
    X_arr = np.array(X_list, dtype=np.float32)
    y_arr = np.array(y_list, dtype=np.float32)[..., None]
    if with_meta:
        return X_arr, y_arr, meta_list
    return X_arr, y_arr

X_train, y_train = make_sliding_windows(train, TIME_STEPS, feature_cols, target_col)
X_valid, y_valid = make_sliding_windows(valid, TIME_STEPS, feature_cols, target_col)

if MAX_TRAIN_WINDOWS:
    X_train = X_train[:MAX_TRAIN_WINDOWS]
    y_train = y_train[:MAX_TRAIN_WINDOWS]

print('train', X_train.shape, y_train.shape)
print('valid', X_valid.shape, y_valid.shape)


## Decoder-Only TFT（簡略版）モデル
- 著者実装の GRN / Interpretable Multi-Head Attention を参考にした軽量構成
- 出力は **relu** でロングオンリーの `score ≥ 0`


In [ ]:
from tensorflow.keras import backend as K


def tf_stack(x, axis=0):
    if not isinstance(x, list):
        x = [x]
    return K.stack(x, axis=axis)


def linear_layer(size, activation=None, use_time_distributed=False, use_bias=True):
    linear = keras.layers.Dense(size, activation=activation, use_bias=use_bias)
    if use_time_distributed:
        linear = keras.layers.TimeDistributed(linear)
    return linear


def apply_gating_layer(x, hidden_layer_size: int, dropout_rate: float = None, use_time_distributed: bool = True, activation=None):
    if dropout_rate is not None:
        x = keras.layers.Dropout(dropout_rate)(x)

    if use_time_distributed:
        activation_layer = keras.layers.TimeDistributed(
            keras.layers.Dense(hidden_layer_size, activation=activation)
        )(x)
        gated_layer = keras.layers.TimeDistributed(
            keras.layers.Dense(hidden_layer_size, activation='sigmoid')
        )(x)
    else:
        activation_layer = keras.layers.Dense(hidden_layer_size, activation=activation)(x)
        gated_layer = keras.layers.Dense(hidden_layer_size, activation='sigmoid')(x)

    return keras.layers.multiply([activation_layer, gated_layer]), gated_layer


def add_and_norm(x_list):
    tmp = keras.layers.Add()(x_list)
    tmp = keras.layers.LayerNormalization()(tmp)
    return tmp


def gated_residual_network(x, hidden_layer_size: int, output_size: int = None, dropout_rate: float = None, use_time_distributed: bool = True, additional_context=None, return_gate: bool = False):
    if output_size is None:
        output_size = hidden_layer_size
        skip = x
    else:
        linear = keras.layers.Dense(output_size)
        if use_time_distributed:
            linear = keras.layers.TimeDistributed(linear)
        skip = linear(x)

    hidden = linear_layer(hidden_layer_size, activation=None, use_time_distributed=use_time_distributed)(x)
    if additional_context is not None:
        hidden = hidden + linear_layer(hidden_layer_size, activation=None, use_time_distributed=use_time_distributed, use_bias=False)(additional_context)
    hidden = keras.layers.Activation('elu')(hidden)
    hidden = linear_layer(hidden_layer_size, activation=None, use_time_distributed=use_time_distributed)(hidden)

    gating_layer, gate = apply_gating_layer(hidden, output_size, dropout_rate=dropout_rate, use_time_distributed=use_time_distributed, activation=None)

    if return_gate:
        return add_and_norm([skip, gating_layer]), gate
    else:
        return add_and_norm([skip, gating_layer])


def get_decoder_mask(self_attn_inputs):
    len_s = tf.shape(self_attn_inputs)[-2]
    bs = tf.shape(self_attn_inputs)[:-2]
    mask = tf.cumsum(tf.eye(len_s, batch_shape=bs), -2)
    return mask

class DecoderMask(keras.layers.Layer):
    def call(self, x):
        len_s = tf.shape(x)[-2]
        bs = tf.shape(x)[:-2]
        return tf.cumsum(tf.eye(len_s, batch_shape=bs), -2)



def tempering_batchdot(input_list):
    d, k = input_list
    temper = tf.sqrt(tf.cast(k.shape[-1], dtype='float32'))
    return K.batch_dot(d, k, axes=[2, 2]) / temper


class ScaledDotProductAttention(keras.layers.Layer):
    def __init__(self, attn_dropout: float = 0.0, **kwargs):
        super().__init__(**kwargs)
        self.dropout = keras.layers.Dropout(attn_dropout)
        self.activation = keras.layers.Activation('softmax')

    def __call__(self, q, k, v, mask):
        attn = keras.layers.Lambda(tempering_batchdot)([q, k])
        if mask is not None:
            mmask = keras.layers.Lambda(lambda x: (-1e9) * (1.0 - tf.cast(x, 'float32')))(mask)
            attn = keras.layers.add([attn, mmask])
        attn = self.activation(attn)
        attn = self.dropout(attn)
        output = keras.layers.Lambda(lambda x: K.batch_dot(x[0], x[1]))([attn, v])
        return output, attn


class InterpretableMultiHeadAttention(keras.layers.Layer):
    def __init__(self, n_head: int, d_model: int, dropout: float, **kwargs):
        super().__init__(**kwargs)
        self.n_head = n_head
        self.d_k = self.d_v = d_k = d_v = d_model // n_head
        self.dropout = dropout

        self.qs_layers = []
        self.ks_layers = []
        self.vs_layers = []

        vs_layer = keras.layers.Dense(d_v, use_bias=False)

        for _ in range(n_head):
            self.qs_layers.append(keras.layers.Dense(d_k, use_bias=False))
            self.ks_layers.append(keras.layers.Dense(d_k, use_bias=False))
            self.vs_layers.append(vs_layer)

        self.attention = ScaledDotProductAttention()
        self.w_o = keras.layers.Dense(d_model, use_bias=False)

    def __call__(self, q, k, v, mask=None):
        n_head = self.n_head
        heads = []
        attns = []
        for i in range(n_head):
            qs = self.qs_layers[i](q)
            ks = self.ks_layers[i](k)
            vs = self.vs_layers[i](v)
            head, attn = self.attention(qs, ks, vs, mask)
            head_dropout = keras.layers.Dropout(self.dropout)(head)
            heads.append(head_dropout)
            attns.append(attn)

        head = keras.layers.Lambda(tf_stack)(heads) if n_head > 1 else heads[0]
        attn = keras.layers.Lambda(tf_stack)(attns)

        outputs = keras.layers.Lambda(K.mean, arguments={'axis': 0})(head) if n_head > 1 else head
        outputs = self.w_o(outputs)
        outputs = keras.layers.Dropout(self.dropout)(outputs)
        return outputs, attn


def sharpe_loss(y_true, y_pred):
    captured = y_true * y_pred
    mean_returns = tf.reduce_mean(captured)
    var = tf.reduce_mean(tf.square(captured)) - tf.square(mean_returns)
    std = tf.sqrt(var + 1e-9)
    return -(mean_returns / std) * tf.sqrt(252.0)


def build_model(time_steps, num_features):
    inputs = keras.Input(shape=(time_steps, num_features), name='inputs')

    # Variable Selection Network (temporal, detailed)
    def temporal_vsn(inputs):
        # inputs: (B, T, F)
        trans_emb_list = []
        for i in range(num_features):
            x_i = inputs[..., i:i+1]
            # per-variable GRN
            trans_emb = gated_residual_network(
                x_i,
                hidden_layer_size=HIDDEN_SIZE,
                output_size=HIDDEN_SIZE,
                dropout_rate=DROPOUT_RATE,
                use_time_distributed=True,
            )
            trans_emb_list.append(trans_emb)
        transformed = keras.layers.Lambda(lambda x: tf.stack(x, axis=-1))(trans_emb_list)
        flatten = keras.layers.Reshape((time_steps, HIDDEN_SIZE * num_features))(transformed)
        # VSN weights (GRN -> softmax)
        mlp_outputs = gated_residual_network(
            flatten,
            hidden_layer_size=HIDDEN_SIZE,
            output_size=num_features,
            dropout_rate=DROPOUT_RATE,
            use_time_distributed=True,
        )
        sparse_weights = keras.layers.Activation('softmax')(mlp_outputs)
        sparse_weights = keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-2))(sparse_weights)
        combined = keras.layers.Multiply()([sparse_weights, transformed])
        temporal_ctx = keras.layers.Lambda(lambda x: tf.reduce_sum(x, axis=-1))(combined)
        return temporal_ctx

    temporal_ctx = temporal_vsn(inputs)
    # LSTM
    lstm_out = keras.layers.LSTM(
        HIDDEN_SIZE,
        return_sequences=True,
        activation='tanh',
        recurrent_activation='sigmoid',
    )(temporal_ctx)

    lstm_out, _ = apply_gating_layer(lstm_out, HIDDEN_SIZE, DROPOUT_RATE, use_time_distributed=True, activation=None)
    temporal_feature = add_and_norm([lstm_out, temporal_ctx])

    # Enrichment
    enriched = gated_residual_network(temporal_feature, HIDDEN_SIZE, dropout_rate=DROPOUT_RATE, use_time_distributed=True)

    # Decoder self-attention
    self_attn_layer = InterpretableMultiHeadAttention(NUM_HEADS, HIDDEN_SIZE, dropout=DROPOUT_RATE)
    mask = DecoderMask()(enriched)
    x, _ = self_attn_layer(enriched, enriched, enriched, mask=mask)
    x, _ = apply_gating_layer(x, HIDDEN_SIZE, DROPOUT_RATE, use_time_distributed=True, activation=None)
    x = add_and_norm([x, enriched])

    decoder = gated_residual_network(x, HIDDEN_SIZE, dropout_rate=DROPOUT_RATE, use_time_distributed=True)
    decoder, _ = apply_gating_layer(decoder, HIDDEN_SIZE, use_time_distributed=True, activation=None)
    transformer_layer = add_and_norm([decoder, temporal_feature])

    outputs = keras.layers.TimeDistributed(
        keras.layers.Dense(1, activation='relu', kernel_constraint=keras.constraints.max_norm(3))
    )(transformer_layer)

    model = keras.Model(inputs, outputs)
    optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE, clipnorm=MAX_GRAD_NORM)
    model.compile(optimizer=optimizer, loss=sharpe_loss)
    return model

model = build_model(TIME_STEPS, len(feature_cols))
model.summary()


## 学習


In [ ]:
callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=EARLY_STOPPING, restore_best_weights=True),
    keras.callbacks.TerminateOnNaN(),
]

history = model.fit(
    X_train,
    y_train,
    validation_data=(X_valid, y_valid),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    shuffle=True,
    callbacks=callbacks,
    verbose=1,
)


## 学習曲線


In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(history.history['loss'], label='train')
ax.plot(history.history['val_loss'], label='val')
ax.set_title('Learning Curve')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss (Sharpe)')
ax.legend()
fig.tight_layout()

learning_curve_path = FIG_DIR / 'learning_curve.png'
fig.savefig(learning_curve_path, dpi=150)
print('Saved:', learning_curve_path)
plt.show()


## テスト予測・ウェイト正規化（合計=1）
- `score ≥ 0` から日次で正規化
- 合計が0の場合は **前日ウェイトを維持**（初日だけ均等配分）


In [ ]:
# テスト用にバッファを付与してスライディング窓を作成
trainvalid_sorted = trainvalid_scaled.sort_values(['ticker', 'date'])

buffered_parts = []
for ticker, g in trainvalid_sorted.groupby('ticker'):
    g = g.sort_values('date')
    tail = g.iloc[-(TIME_STEPS-1):] if len(g) >= (TIME_STEPS-1) else g
    tpart = test[test['ticker'] == ticker].sort_values('date')
    buffered_parts.append(pd.concat([tail, tpart], ignore_index=True))

buffered = pd.concat(buffered_parts, ignore_index=True)

X_test, y_test, meta = make_sliding_windows(buffered, TIME_STEPS, feature_cols, target_col, with_meta=True)
print('test windows', X_test.shape, y_test.shape)

# 予測
pred = model.predict(X_test, batch_size=256)
score_last = pred[:, -1, 0]
true_last = y_test[:, -1, 0]

# メタ情報をDataFrame化
rows = []
for (ticker, date, y_last), score in zip(meta, score_last):
    rows.append({'ticker': ticker, 'date': pd.to_datetime(date), 'score': float(score), 'target_returns': float(y_last)})

pred_df = pd.DataFrame(rows)
# テスト期間のみ残す
pred_df = pred_df[(pred_df['date'].dt.year >= TEST_BOUNDARY) & (pred_df['date'].dt.year < TEST_END)].copy()

print(pred_df.head())


In [ ]:
def normalize_weights(df_scores: pd.DataFrame) -> pd.DataFrame:
    df_scores = df_scores.sort_values(['date', 'ticker']).reset_index(drop=True)
    dates = df_scores['date'].sort_values().unique()

    prev_weights = None
    out_rows = []

    for d in dates:
        sub = df_scores[df_scores['date'] == d].copy()
        scores = sub['score'].to_numpy()
        score_sum = np.nansum(scores)

        if score_sum > 0:
            weights = scores / score_sum
        else:
            if prev_weights is not None:
                weights = np.array([prev_weights.get(t, 0.0) for t in sub['ticker']])
            else:
                # 初日は均等配分
                weights = np.ones(len(sub)) / len(sub)

        sub['weight'] = weights
        prev_weights = dict(zip(sub['ticker'], sub['weight']))
        out_rows.append(sub)

    return pd.concat(out_rows, ignore_index=True)

weights_df = normalize_weights(pred_df)

# 合計チェック
check = weights_df.groupby('date')['weight'].sum().describe()
print(check)


In [ ]:
def compute_portfolio_returns(weighted_df: pd.DataFrame, cost_bps: float = 5.0) -> pd.DataFrame:
    # ポートフォリオ日次リターン
    weighted_df = weighted_df.copy()
    weighted_df['weighted_return'] = weighted_df['weight'] * weighted_df['target_returns']
    gross = weighted_df.groupby('date')['weighted_return'].sum().rename('gross_return')

    # ターンオーバーとコスト
    w_pivot = weighted_df.pivot(index='date', columns='ticker', values='weight').sort_index()
    turnover = 0.5 * w_pivot.diff().abs().sum(axis=1).fillna(0.0)
    cost = (cost_bps * 1e-4) * turnover

    net = gross - cost
    out = pd.DataFrame({'gross_return': gross, 'cost': cost, 'net_return': net})
    return out

portfolio = compute_portfolio_returns(weights_df, cost_bps=5.0)
print(portfolio.head())


## ベースライン（TSMOM / MACD）
- ロングオンリー
- 合計ウェイト=1、ゼロ時は前日維持
- 同一の取引コスト（5bps）


In [ ]:
def build_baseline_scores(features_df: pd.DataFrame, method: str) -> pd.DataFrame:
    df = features_df.copy()
    df = df[(df['year'] >= TEST_BOUNDARY) & (df['year'] < TEST_END)].copy()

    if method == 'TSMOM':
        df['score'] = (df['raw_annual_return'] > 0).astype(float)
    elif method == 'MACD':
        df['score'] = np.maximum(df['macd_score'].values, 0.0)
    else:
        raise ValueError('Unknown method')

    return df[['ticker', 'date', 'score', 'target_returns']]

baseline_tsmom = build_baseline_scores(features, 'TSMOM')
baseline_macd = build_baseline_scores(features, 'MACD')

# 予測と同じ日付・ティッカーに合わせる
pred_keys = pred_df[['date', 'ticker']].drop_duplicates()
baseline_tsmom = baseline_tsmom.merge(pred_keys, on=['date','ticker'], how='inner')
baseline_macd = baseline_macd.merge(pred_keys, on=['date','ticker'], how='inner')

w_tsmom = normalize_weights(baseline_tsmom)
w_macd = normalize_weights(baseline_macd)

port_tsmom = compute_portfolio_returns(w_tsmom, cost_bps=5.0)
port_macd = compute_portfolio_returns(w_macd, cost_bps=5.0)


## 指標計算


In [ ]:
def compute_metrics(returns: pd.Series) -> dict:
    returns = returns.dropna()
    if len(returns) == 0:
        return {}

    ann_return = (1 + returns).prod() ** (252.0 / len(returns)) - 1
    ann_vol = returns.std() * np.sqrt(252.0)
    sharpe = (returns.mean() / returns.std()) * np.sqrt(252.0) if returns.std() > 0 else np.nan

    cum = (1 + returns).cumprod()
    peak = cum.cummax()
    drawdown = (cum / peak) - 1.0
    max_dd = drawdown.min()
    calmar = ann_return / abs(max_dd) if max_dd < 0 else np.nan

    downside = returns[returns < 0]
    downside_std = downside.std() * np.sqrt(252.0) if len(downside) > 0 else np.nan
    sortino = (returns.mean() / downside_std) * np.sqrt(252.0) if downside_std and downside_std > 0 else np.nan

    return {
        'annual_return': ann_return,
        'annual_volatility': ann_vol,
        'sharpe_ratio': sharpe,
        'max_drawdown': max_dd,
        'calmar_ratio': calmar,
        'sortino_ratio': sortino,
    }

metrics = pd.DataFrame({
    'MomentumTransformer': compute_metrics(portfolio['net_return']),
    'TSMOM': compute_metrics(port_tsmom['net_return']),
    'MACD': compute_metrics(port_macd['net_return']),
})

metrics_path = TABLE_DIR / 'performance_metrics.csv'
metrics.to_csv(metrics_path)
print(metrics)
print('Saved:', metrics_path)


## 累積リターンの可視化


In [ ]:
def cumulative_returns(srs: pd.Series) -> pd.Series:
    return (1 + srs).cumprod()

cum_model = cumulative_returns(portfolio['net_return'])
cum_tsmom = cumulative_returns(port_tsmom['net_return'])
cum_macd = cumulative_returns(port_macd['net_return'])

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(cum_model.index, cum_model.values, label='MomentumTransformer')
ax.plot(cum_tsmom.index, cum_tsmom.values, label='TSMOM')
ax.plot(cum_macd.index, cum_macd.values, label='MACD')
ax.set_title('Cumulative Returns (Net, 5bps)')
ax.set_xlabel('Date')
ax.set_ylabel('Cumulative Return')
ax.legend()
ax.grid(True, alpha=0.3)
fig.tight_layout()

fig_path = FIG_DIR / 'cumulative_returns.png'
fig.savefig(fig_path, dpi=150)
print('Saved:', fig_path)
plt.show()


## まとめ
- ロングオンリー + 合計ウェイト=1 の制約を満たすよう、推論後に日次正規化を実施。
- 取引コストは 5 bps のみを適用。
- 学習曲線と累積リターンを `outputs/figures/` に保存。

必要に応じて以下を調整してください:
- `MAX_TRAIN_WINDOWS`（学習サンプル数の制限）
- `HIDDEN_SIZE` / `DROPOUT_RATE` / `NUM_HEADS`
- `EPOCHS` / `BATCH_SIZE`
